In [ ]:
import numpy as np
from scipy.optimize import linprog
import requests
from datetime import datetime, timezone


PM_TARGET     = 15.0
WHO_GUIDE     = 5.0
PULL_FRACTION = 0.4
SCALE         = 10.0

EFFECTS = np.array([3.0, 5.0, 2.5])    
BASE_COST = np.array([200.0, 250.0, 175.0])
P_MAX   = 0.7                           
UB      = np.array([40.0, 30.0, 50.0])  
SLACK_PEN = 1e4                         

def geocode_city_openmeteo(city):
    r = requests.get("https://geocoding-api.open-meteo.com/v1/search", params={"name": city, "count": 1, "language": "en", "format": "json"}, timeout=15)
    r.raise_for_status()
    items = (r.json().get("results") or [])
    if not items: return None
    it = items[0]
    return {"lat": it["latitude"], "lon": it["longitude"], "name": it.get("name", city)}

def fetch_pm25_from_openmeteo(city):
    g = geocode_city_openmeteo(city)
    if not g: return None
    r = requests.get("https://air-quality-api.open-meteo.com/v1/air-quality", params={"latitude": g["lat"], "longitude": g["lon"], "hourly": "pm2_5", "timezone": "UTC"}, timeout=15)
    r.raise_for_status()
    js = r.json(); hourly = js.get("hourly") or {}
    times = hourly.get("time") or []; vals = hourly.get("pm2_5") or []
    if not times or not vals: return None
    return {"city": city, "pm25": float(vals[-1]), "unit": "µg/m³", "when_utc": times[-1],
            "source": "Open-Meteo", "location_name": g["name"]}

def fetch_air_quality(city):
    try:
        d = fetch_pm25_from_openmeteo(city)
        if d: return d
    except Exception:
        pass
    return None

def required_units_from_pm(pm):
    if pm >= PM_TARGET:
        gap = pm - PM_TARGET
        return max(1.0, gap * SCALE)
    else:
        gap_to_who = max(0.0, pm - WHO_GUIDE)
        return max(1.0, PULL_FRACTION * gap_to_who * SCALE)
def costs_for_city(city, base = BASE_COST, city_multipliers: dict | None = None):
    m = 1.0
    if city_multipliers and city in city_multipliers:
        m = float(city_multipliers[city])
    return (base * m).astype(float)

def build_compact_standard_form(required_units, costs, ub):
    e1, e2, e3 = EFFECTS
    P = P_MAX

    A = np.zeros((4, 7))
    b = np.zeros(4)

    A[0, 0] = e1; A[0, 1] = e2; A[0, 2] = e3; A[0, 3] = 1.0
    b[0] = required_units

    A[1, 0] = e1 * (1.0 - P); A[1, 1] = -P * e2; A[1, 2] = -P * e3; A[1, 4] = 1.0

    A[2, 0] = -P * e1; A[2, 1] = e2 * (1.0 - P); A[2, 2] = -P * e3; A[2, 5] = 1.0

    A[3, 0] = -P * e1; A[3, 1] = -P * e2; A[3, 2] = e3 * (1.0 - P); A[3, 6] = 1.0

    c = np.zeros(7)
    c[0:3] = costs
    c[3]   = SLACK_PEN

    bounds = [(0, ub[0]), (0, ub[1]), (0, ub[2]), (0, None), (0, None), (0, None), (0, None)]
    print(A)
    return c, A, b, bounds


def fmt_utc(s):
    if not s: return "n/a"
    try:
        dt = datetime.fromisoformat(s.replace("Z", "+00:00"))
        return dt.astimezone(timezone.utc).strftime("%Y-%m-%d %H:%M UTC")
    except Exception:
        return str(s)

def run(city):
    print("\n" + "="*60)
    print("CARBON REDUCTION — COMPACT STANDARD FORM (small A)")
    print("="*60 + "\n")

    aq = fetch_air_quality(city)
    if not aq:
        print(f"No air quality data for '{city}'"); return

    pm = aq["pm25"]
    print(f"Source: {aq['source']}")
    print(f"City/Location: {aq['city']} — {aq['location_name']}")
    print(f"PM2.5: {pm:.2f} {aq['unit']}  (as of {fmt_utc(aq['when_utc'])})")

    R = required_units_from_pm(pm)
    costs = costs_for_city(aq["city"])

    print(f"\nTarget (reduction units): {R:.2f}"f"[toward {'15' if pm>=PM_TARGET else 'WHO 5'}]")
    print(f"Diversification cap P_MAX: {P_MAX:.0%}  |  Costs used: {costs.round(2)}")

    c, A, b, bounds = build_compact_standard_form(R, costs, UB)

    print(f"\nStandard form (compact):  A ∈ R^{{{A.shape[0]}×{A.shape[1]}}},  b ∈ R^{A.shape[0]},  x ∈ R^{A.shape[1]}")

    res = linprog(c, A_eq=A, b_eq=b, bounds=bounds, method="highs")
    if not res.success:
        print(f"LP failed: {res.message}"); return

    # Unpack
    x1, x2, x3, v, s1, s2, s3 = res.x
    t = EFFECTS @ np.array([x1, x2, x3])   # achieved reduction (units)
    cost_val = costs @ np.array([x1, x2, x3])
    delta_pm = t / SCALE
    final_pm = max(0.0, pm - delta_pm)

    print("\n✓ OPTIMAL SOLUTION FOUND\n")
    print("Decision variables:")
    print(f"Transport (x1):   {x1:7.3f}")
    print(f"Industry  (x2):   {x2:7.3f}")
    print(f"  Residential(x3):   {x3:7.3f}")
    print(f"  Shortfall slack (v): {v:7.3f}  (0 means target met)")

    contrib = np.array([EFFECTS[0]*x1, EFFECTS[1]*x2, EFFECTS[2]*x3])
    shares = (contrib / t * 100.0) if t > 1e-12 else np.zeros_like(contrib)

    print("\nConstraint slacks (diversification):")
    print(f"  s1={s1:.4f}, s2={s2:.4f}, s3={s3:.4f}  (≈0 means the sector’s share cap is binding)")

    print("\nAir-quality change:")
    print(f"  Start PM2.5: {pm:.2f} µg/m³")
    print(f"  Reduction:   {delta_pm:.2f} µg/m³")
    print(f"  Final PM2.5: {final_pm:.2f} µg/m³")

    print("\nResults:")
    print(f"  Achieved reduction (t): {t:6.2f}   Required: {R:6.2f}")
    print(f"  Intervention cost:${cost_val:,.2f}")
    print(f"  Sector shares:Transport {shares[0]:.1f}% | Industry {shares[1]:.1f}% | Residential {shares[2]:.1f}%")
    print(f"  Objective value:{res.fun:,.2f}  (= cost + {SLACK_PEN:g}·v)")

if __name__ == "__main__":
    while True:
        try:
            city = input("\nEnter a city (or 'q' to quit): ").strip()
        except (EOFError, KeyboardInterrupt):
            print("\nGoodbye!"); break
        if city.lower() in {"q","quit","exit"}:
            print("\nGoodbye!"); break
        if city:
            run(city)
        else:
            print("Please enter a valid city name.")



CARBON REDUCTION — COMPACT STANDARD FORM (small A)

Source: Open-Meteo
City/Location: London — London
PM2.5: 4.60 µg/m³  (as of 2025-11-16 04:00 UTC)

Target (reduction units): 1.00[toward WHO 5]
Diversification cap P_MAX: 70%  |  Costs used: [200. 250. 175.]
[[ 3.    5.    2.5   1.    0.    0.    0.  ]
 [ 0.9  -3.5  -1.75  0.    1.    0.    0.  ]
 [-2.1   1.5  -1.75  0.    0.    1.    0.  ]
 [-2.1  -3.5   0.75  0.    0.    0.    1.  ]]

Standard form (compact):  A ∈ R^{4×7},  b ∈ R^4,  x ∈ R^7

✓ OPTIMAL SOLUTION FOUND

Decision variables:
Transport (x1):     0.100
Industry  (x2):     0.140
  Residential(x3):     0.000
  Shortfall slack (v):   0.000  (0 means target met)

Constraint slacks (diversification):
  s1=0.4000, s2=0.0000, s3=0.7000  (≈0 means the sector’s share cap is binding)

Air-quality change:
  Start PM2.5: 4.60 µg/m³
  Reduction:   0.10 µg/m³
  Final PM2.5: 4.50 µg/m³

Results:
  Achieved reduction (t):   1.00   Required:   1.00
  Intervention cost:$55.00
  Sector sha